### 决策树  
    决策树的思想是通过数据集的各种属性,通过一种由各种子决策组成的树状决策来学习模型,从而对数据集进行分类.
算法如下:
___
**输入**: 训练集 $D=\{(x_1,y_1),(x_2,y_2),...,(x_m,y_m)\}$  
     属性集$A=\{a_1,a_2,...,a_d\}$  
**过程**:函数TreeGenerate(D,A)  
1:&emsp;生成节点node;(通过划分选择来进行节点生成)    
2:**if** $D$中样本全属于同一类别$C$**then**(即样本没有区别)  
3:&emsp;将node标记为$C$类叶节点;**return**   
4:**end if**  
5:**if** $A=\emptyset$**OR**$D$中样本在$A$上取值相同**then**  
6:&emsp;将node标记为叶节点,其类别标记为$D$中样本数最多的类;**return**  
7:**end if**  
8:从$A$中选择最优化分属性$a_*$;  
9:**for** $a_*$的每一个值$a_*^v$**do**  
10:&emsp;为node生成一个分支;令$D_v$表示$D$中在$a_*$上取值为$a_*^v$的样本集;  
11:&emsp;**if**$D_v$为空**then**  
12:&emsp;&emsp;将分支节点标记为叶节点,其类别标记为$D$中样本最多的类;**return**  
13:&emsp;**else**  
14:&emsp;&emsp;以TreeGenerate($D_v,A\backslash \{a_*\})$为分支节点  
15:&emsp;**end if**  
16:**end for**  
**输出:**以node为根节点的一棵决策树
***

由上述算法可以看出决策树的第8行是关键,即如何选择最优的属性划分.一般我们希望节点包含的样本应尽可能的属于同一种类型,即节点的纯度越来越高.
#### 划分选择
既然节点的纯度越高越好,那么用什么来衡量样本的纯度呢?  
信息熵(information entropy)是度两样本集合纯度最常用的一种指标.假定当前样本集合$D$中第$k$类样本所占的比例为$p_k(k=1,2,...,|\cal Y|)$,则$D$的信息熵为
$$Ent(D)=-\sum_{k=1}^{|\cal Y|}p_klog_2p_k.$$
$Ent(D)$的值越小,则$D$的纯度越高.(从公式可以看出信息熵是各分类概率与概率对数乘积的和,最大为$|\cal Y|$即全部为一类,另一种是全都不是为0)  
假定离散属性$a$有V个可能的取值$\{a^1,a^2,...,a^v\}$,若使用$a$来进行划分,则会产生V个节点,其中第$v$个节点包含了$D$中所有在属性$a$上取值为$a^v$的样本,记为$D^v$.于是我们可以计算出$D^v$的信息熵,在考虑不同分支节点所包含的样本数不同,给分支节点赋予权重$|D^v|/|D|$,于是我们可以就可以计算以$a$为节点对样本进行划分所获得的信息增益(information gain):
$$Gain(D,a)=Ent(D)-\sum_{v=1}^{V}\frac {|D^v|}{|D|}Ent(D^v)$$
信息增益从公式可以看出是样本的信息熵减去以某一属性划分的信息熵与其权重的乘积.可以理解为所选属性对原始数据增加的信息.  
所以我们可以以信息增益最大的属性来进行划分,但同样会有一些问题,后面会讨论.  
同样可以想象如果一个属性的取值几乎等同于样本的数量,其信息熵同样很大但是却没有实际意义,所以我们需要去除这些属性.  
我们引入一个新的度量信息增益率:
$$Gain_ratio(D,a)=\frac {Gain(D,a)}{IV(a)}$$
其中$$IV(a)=-\sum_{v=1}^{V}\frac {|D^v|}{|D|}log_2\frac {|D^v|}{|D|}$$
增益率越大说明属性的取值越少,综合两种度量我们在选择划分时可以,先选出高出平均增益的属性,然后在在其中选择信息增益率比较高的.

#### 基尼指数
基尼指数是另一种数据集纯度的度量,公式为:
$$Gini(D)=\sum_{k=1}^{|\cal Y|}\sum_{k'\neq k}p_kp_k'$$
$$=1-\sum_{k=1}^{|\cal Y|}p_k^2.$$
直观来说$Gini(D)$反映了从数据集$D$中最忌抽取两个样本,其类别标记不一致的概率.所以其值越小说明数据集越纯.
属性a的基尼指数定义为:
$$Gini_index(D,a)=\sum_{v=1}{V}\frac {|D^v|}{|D|}Gini(D^v).$$
于是我们在候选属性集合$A$中,选择基尼指数最小的属性.

#### 剪枝处理
剪枝处理简单来说就是为了解决过拟合问题,通过主动去掉一些分支以降低过拟合的风险.  
剪枝处理有两种基本策略:预剪枝和后剪枝.  
预剪枝是在决策树生成过程中,对每个节点在划分前进行估计,若当前节点的划分不能带来决策树泛化性能的提升,则停止划分并将其标记为叶节点.  
后剪枝则是先从训练集生成一颗完整的决策树,然后自下而上的对非叶节点进行考察,若将该节点对应的子树(或者叫枝杈)替换为叶节点能够提升整体泛化性能,则将该子树替换为叶节点.  
